In [1]:
import tensorflow as tf
import numpy as np
import data as d
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
d.recompute()

In [2]:
def precision(predictions, labels):
    p = [np.argmax(x) for x in predictions]
    labs = [np.argmax(x) for x in labels]
    return 100 * precision_score(labs, p, average='macro')
def confusion(predictions, labels):
    p = [np.argmax(x) for x in predictions]
    labs = [np.argmax(x) for x in labels]
    return confusion_matrix(labs, p)

In [3]:
graph = tf.Graph()
with graph.as_default():
    learning_rate = 0.001
    display_step = 1
    bacthes_per_epoch = 3000
    # Network Parameters
    training_epochs = 3
    n_input = 1000
    n_hidden_1 = 2000
    n_hidden_2 = 500
    n_hidden_3 = 50
    n_hidden_4 = 1000
    n_classes = 5
    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_classes])
    c_w = tf.placeholder(tf.float32, [250])
#     tf_valid_dataset = tf.constant(valid_dataset)
    dropout_rate = tf.placeholder(tf.float32)
    
    def multilayer_perceptron(X, weights, biases):
        layer_1 = tf.nn.relu(
            tf.add(
                tf.matmul(tf.nn.dropout(X, dropout_rate), weights['h1']),
                biases['b1'])
            )
        layer_2 = tf.nn.relu(
            tf.add(
                tf.matmul(tf.nn.dropout(layer_1, dropout_rate), weights['h2']),
                biases['b2'])
            )
        layer_3 = tf.nn.relu(
            tf.add(
                tf.matmul(tf.nn.dropout(layer_2, dropout_rate), weights['h3']),
                biases['b3'])
            )
        return tf.add(
            tf.matmul(tf.nn.dropout(layer_3, dropout_rate), weights['out']),
            biases['out']
            )
    
    
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
        'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    # Construct model
    pred = multilayer_perceptron(x, weights, biases)
    prediction = tf.nn.softmax(pred)
    # Define loss and optimizer
    total_l2_loss = 0
    for k, v in weights.items():
        total_l2_loss += tf.nn.l2_loss(v)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y) * c_w) + 0.000004 * (total_l2_loss)
#     cost = precision(prediction, y)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    init = tf.initialize_all_variables()
    
#     valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset))

In [ ]:
prev_loss = 100000
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    sess.run(init)
    saver = tf.train.Saver()
    for epoch in range(training_epochs):
        for i in range(bacthes_per_epoch):
            batch_xs, batch_ys, cw = d.train_batch_data(50)
            train_feed_dict = {
                x: batch_xs,
                y: batch_ys,
                dropout_rate: 9.0,
                c_w: cw
            }

            if i % 500 == 0:
                test_feed_dict = {
                    x: batch_xs,
                    y: batch_ys,
                    dropout_rate: 1.0,
                    c_w: cw
                }
                _, c = sess.run(
                    [optimizer, cost], feed_dict=train_feed_dict
                )
#                 if abs(prev_loss - c) < 0.0001:
#                     break
#                 prev_loss = c
                print "Minibatch loss at step", i, ":", c
                batch_precision = precision(prediction.eval(feed_dict=test_feed_dict),batch_ys)
                print "Minibatch precision at step", i, ":", batch_precision
#                 validation_precision = precision(
#                     valid_prediction.eval(feed_dict=test_feed_dict),
#                     valid_labels
#                 )
#                 print(
#                     "Validation precission at step", step, ":",
#                     validation_precision,
#                     end='\n',
#                     flush=True
#                 )
            else:
                _, c = sess.run([optimizer, cost], feed_dict=train_feed_dict)
#                 if abs(prev_loss - c) < 0.0001:
#                     break
#                 prev_loss = c
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1))
            
    #Test model on test dataset
    
    tdata = np.load('out/new_input.npz')['test']
    a, b = tdata[:, :-1], tdata[:, -1]
    b = LabelBinarizer().fit_transform(b-1)
    preds = prediction.eval(feed_dict={x:a,y:b,dropout_rate:1.0})
    print("Test Precision:", precision(preds, b))
    print(confusion(preds, b))  